# Generating graph data

In [1]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interactive

All graph generators can be found [here](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html).

In [2]:
def f(n,k,p):
    g = nx.newman_watts_strogatz_graph(n,k,p)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), k=(0,10), p=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), IntSlider(value=5, description='k', max=10…

In [3]:
def f(n,p):
    g = nx.erdos_renyi_graph(n,p)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), p=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=0.5, description='p', ma…

In [4]:
def f(n,p1,p2):
    g = nx.random_lobster(n,p1,p2)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), p1=(0.0,1.0), p2=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=0.5, description='p1', m…

In [5]:
def f(n,p):
    g = nx.duplication_divergence_graph(n,p)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), p=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=0.5, description='p', ma…

In [6]:
def f(N,n,p,q):
    g = nx.partial_duplication_graph(N,n,p,q)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, N=(8,25), n=(0,7), p=(0.0,1.0), q=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=16, description='N', max=25, min=8), IntSlider(value=3, description='n',…

In [7]:
def f(n,radius):
    g = nx.duplication_divergence_graph(n,radius)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), radius=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=0.5, description='radius…

In [8]:
def f(n,alpha,beta):
    g = nx.waxman_graph(n,alpha,beta)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), alpha=(0.0,1.0), beta=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=0.5, description='alpha'…

In [9]:
def f(n,p,q):
    g = nx.navigable_small_world_graph(n,p,q)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,5), p=(1,5), q=(0,5))
interactive_plot

interactive(children=(IntSlider(value=2, description='n', max=5), IntSlider(value=3, description='p', max=5, m…

In [10]:
def f(l,k):
    g = nx.connected_caveman_graph(l,k)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, l=(1,6), k=(1,10))
interactive_plot

interactive(children=(IntSlider(value=3, description='l', max=6, min=1), IntSlider(value=5, description='k', m…

In [11]:
def f(n,s,v,p_in,p_out):
    g = nx.gaussian_random_partition_graph(n,s,v,p_in, p_out)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), s=(0.0,5.0), v=(0.1,2.0), p_in=(0.0,1.0), p_out=(0.0,1.0))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), FloatSlider(value=2.5, description='s', ma…

In [12]:
def f(n,k):
    g = nx.windmill_graph(n,k)
    nx.draw(g)
    plt.show()
    
interactive_plot = interactive(f, n=(0,25), k=(1,7))
interactive_plot

interactive(children=(IntSlider(value=12, description='n', max=25), IntSlider(value=4, description='k', max=7,…